# 31-collate-xml-entities-spans
> Structuring the data to be compatible with Spacy

The code below takes as input two separate dataframes, one containing parsed volume XML from the Spatial Historian and the other containing entity references, also from Spatial Historian. The input dataframes are merged using folio IDs as an index, then entry text is searched for the referenced entities to generate spans defining their location. The final output dataframe contains the following columns: vol_id, vol_titl, fol_id, entry_no, text (these first five from parsed XML), entity, label (from entity references), start, and end (for spans, generated by this code).

In [ ]:
#default_exp collate

In [ ]:
#export

import pandas as pd
import numpy as np

## Demo dummy data
The expected dataframe formats to be passed in appear as below.  One is the parsed XML from Spatial Historial.  The other contains entity references.

In [ ]:
#no_test
volume_ids = ["1", "1", "1", "2", "3"]
volume_titles = ["First book"] * 3 + ["Second book", "Third book"]
folio_ids = ["101", "102", "102", "103", "104"]
entry_numbers = ["101-1", "102-1", "102-2", "103-1", "104-1"]
texts = ["Daniel wrote some code.", "Tyrion ate some food and Tyrion drank some water.", "Tyrion went to sleep while Daniel kept working.", "The rabbit ran away.", "One day the rabbit would be eaten by Tyrion."]
names = ["Tyrion", "Daniel", "rabbit"]
types = ["Person"] * len(names)
sources = ["skhgrhb-0102; hfiheiuhf-0104", "fkheuihfi-0101; aehuihifh-0102", "hfuiehlfiw-0103; afefafaef-0104"]

xml_dict = {"vol_id": volume_ids, "vol_titl": volume_titles, "fol_id": folio_ids, "entry_no": entry_numbers, "text": texts}
entity_dict = {"Entity Type": types, "Name": names, "Source Associator": sources}

xml_df = pd.DataFrame(xml_dict)
ent_df = pd.DataFrame(entity_dict)

In [ ]:
#no_test
xml_df

,vol_id,vol_titl,fol_id,entry_no,text
0,1,First book,101,101-1,Daniel wrote some code.
1,1,First book,102,102-1,Tyrion ate some food and Tyrion drank some water.
2,1,First book,102,102-2,Tyrion went to sleep while Daniel kept working.
3,2,Second book,103,103-1,The rabbit ran away.
4,3,Third book,104,104-1,One day the rabbit would be eaten by Tyrion.


In [ ]:
#no_test
ent_df

,Entity Type,Name,Source Associator
0,Person,Tyrion,skhgrhb-0102; hfiheiuhf-0104
1,Person,Daniel,fkheuihfi-0101; aehuihifh-0102
2,Person,rabbit,hfuiehlfiw-0103; afefafaef-0104


## Generating spans for Spacy input
Spacy requires that the integer location of the entities within the texts are known and identified for the training set.  With the entry `Chickpea is a dog`, the spans returned should return the start and ending points (specifically one past the ending location - Pythonic referencing) of the entity.  Here, the entity is `Chickpea`, and the start location should be 0, and the end location should be 8.  The following two functions achieves this for the Spatial Historian entities.

`genAltEnts` additionally checks for anomalies in representation of the data in Spatial Historian.  Sometimes, an entity `Chickpea` may appear as `C hickpea` or `Ch#ckpea` if certain parts of the name are ambiguous.  This function checks for these scenarios by generating all possible forms of the entity with spaces and pound signs inserted.

In [ ]:
#export

def genAltEnts(entity):
    '''
    Function genAltEnts: This function takes an entity and generates all possible forms of it with errant spaces or pound
    symbols mixed in.
        Inputs: entity: String of entity
        Output: list of possible forms of the entity with spaces and pound signs inserted
    '''
    alt_ents = []
    
    #individual characters replaced by ' '
    for i in range(1, len(entity)):
        alt_ents.append((entity[:i]) + ' ' + entity[i:])
            
    #individual characters replaced by #
    for j in range(1, len(entity)):
        alt_ents.append((entity[:j]) + '#' + entity[j + 1:])
    
    #missing last name
    #if (len(entity.split(' ')) > 1):
        #alt_ents.append(entity[:entity.rfind(' ')])
    
    return alt_ents

In [ ]:
# export
#search entry for each instance of an entity reference

def generateSpans(entry, entity):
    '''
    Function generateSpans: This function takes individual entries and entities from the merged dataframes and 
    returns a list containing a span defining each instance in which that entity appears in that entry 
    (spans are returned as nested lists).
        Inputs: entry: String of entry text
                entity: String of entity whose span is to be located within the text
        Output: nested list of spans for the entity for each place it appears within the entry.
    '''
    curr_index = 0
    spans = []
    entity_len = len(entity)    
   
    #Scroll through each entry and find each instance of the entity and append spans if not already present
    while entry.find(entity, curr_index) != -1:
        entity_start = entry.find(entity, curr_index)
        span = [entity_start, entity_start + entity_len]
        curr_index = span[1]
        if span not in spans:
            spans.append(span)
            
    #If no spans are present        
    if spans == []:
        alts = genAltEnts(entity)
        for alt in alts:
            alt_len = len(alt)
            while entry.find(alt, curr_index) != -1:
                entity_start = entry.find(alt, curr_index)
                span = [entity_start, entity_start + alt_len]
                curr_index = span[1]
                if span not in spans:
                    spans.append(span)
    
    return spans

### Testing: `generateSpans`

In [ ]:
#no_test
#expected output: [[15, 19]]

print(generateSpans("This is just a test!", "test"))

[[15, 19]]


In [ ]:
#no_test
#expected output: [[8, 10], [11, 13]]
#Repeated entities are a fairly rare corner case for our purposes, but I wanted to account for it.

print(generateSpans("What is it it appears multiple times?", "it"))

[[8, 10], [11, 13]]


In [ ]:
#no_test
#expected output: []
#This (entities not appearing in the entry) should not happen, but if so the desired behavior is not to build a row in the final
#dataframe.

print(generateSpans("It isn't here!", "fish"))

[]


## Getting the data into the correct format

These functions take the input dataframes and convert them into lists for further processing (it's likely completely possible to do this all using dataframes, but I'm more used to interacting with lists). parseEntities takes the entity reference dataframe as input and expects three columns labeled "Entity Type", "Name", and "Source Associator". parseXML takes the parsed XML dataframe as input and expects five columns labeled "vol_id", "vol_titl", "fol_id", "entry_no", and "text".

In [ ]:
# export
def parseEntities(df_entities):
    '''
    Function parseEntities: takes entity reference dataframe as input, returns each column as a list for ease of processing
    and boils source references down to unique portion
        Inputs: df_entities: pandas DataFrame with columns "Entity Type", "Name", and "Source Associator"
        Outputs: 3 lists: entity types, entities, and parsed source associators
    '''
    types = df_entities["Entity Type"].tolist()
    names = df_entities["Name"].tolist()
    
    sources = []
    for source_assoc in df_entities["Source Associator"]:
        temp = []
        refs = source_assoc.split(';')
        for ref in refs:
            source = ref[ref.find('-') + 1:]
            while source[0] == '0':
                source = source[1:]
            if (not source in temp):
                temp.append(source)
        sources.append(temp)
    
    return types, names, sources

In [ ]:
# export
def parseManualEntities(csv_entities):
    '''
    Function parseManualEntities: takes manually produced entity reference csv as input and produces same outputs
    as those of parseEntities
    Inputs: csv_entities: path to csv containing two columns, one with folio ids and one with a list of names appearing on
    each folio, separated by semicolons
    Outputs: 3 lists: entity types, entities, and parsed source associatos
    '''
    csv = open(csv_entities, 'r', encoding="utf-8")
    folios = []
    people = []
    
    for line in csv:
        if line[-1] == '\n':
            line = line[:-1]
        if "Folio" in line:
            continue
        folio, temp = line.split(',')
        ppl = temp.split(';')
        folios.append(folio)
        people.append(ppl)
    
    csv.close()
    
    names = []
    sources = []
    
    for x in range(len(folios)):
        for person in people[x]:
            if not (person in names):
                names.append(person)
                temp = [folios[x]]
                sources.append(temp)
            else:
                loc = names.index(person)
                if not (folios[x] in sources[loc]):
                    sources[loc].append(folios[x])
                
    types = ["PER"] * len(names)
    
    return types, names, sources

In [ ]:
# export
def parseXML(df_XML):
    '''
    Function parseXML: takes volume XML dataframe as input and returns each column as a list
        Inputs: df_XML: pandas DataFrame with columns "vol_id", 'vol_titl', 'fol_id', 'entry_no', and 'text'
        Outputs: 5 lists corresonding to each column: volume ids, volume titles, folio ids, entry numbers, and entry texts
    '''
    volume_ids = df_XML["vol_id"].tolist()
    volume_titles = df_XML["vol_titl"].tolist()
    folio_ids = df_XML["fol_id"].tolist()
    entry_numbers = df_XML["entry_no"].tolist()
    entry_texts = df_XML["text"].tolist()
    
    return volume_ids, volume_titles, folio_ids, entry_numbers, entry_texts

The `genEntryIDs` function returns a list of the IDs for all entries in which an input entity appears on an input folio. It also takes as input the full lists of folio IDs, entry IDs, and entry texts from the parsed XML.

In [ ]:
# export
def genEntryIDs(folio_ids, entry_ids, texts, entity, entity_folio):
    '''
    Function genEntryIDs: returns of list of IDs for all entries in which an input entity appears in a folio.
        Inputs: Sets of lists, where the index of each list correponds to the information in every other list index (i.e., each list is a column of a dataframe)
                folio_ids: list of folio IDs to be processed
                entry_ids: list of IDs for each entry
                texts: list of entry texts 
                entity: entity to be found in the data
                entity_folio: folio in which to look for the entity
        Outputs: 5 lists corresonding to each column: volume ids, volume titles, folio ids, entry numbers, and entry texts
    '''
    
    #find the indices of the entity folio of interest
    np_fol = np.array(folio_ids)
    folio_indices = np.where(np_fol == entity_folio)[0]    
    
    matches = []
    
    #Determine if the entity is in the folios if interest and append entry_id if so
    for entries in folio_indices:
        if texts[entries].find(entity) != -1:
            matches.append(entry_ids[entries])
            
    #Check the to see if maybe the entity has unusual representation with # or ' ' inserted     
    if matches == []:
        alts = genAltEnts(entity)
        for alt in alts:
            for entries in folio_indices:
                if (texts[entries].find(alt) != -1) and (entry_ids[entries] not in matches):
                    matches.append(entry_ids[entries])                    
    
    return matches

### Testing: `genEntryIDs`

In [ ]:
#no_test
#dummy data to quickly test genEntryIDs

folio_ids = ["1", "1", "1818", "1818"]
entry_ids = ["1-1", "1-2", "1818-1", "1818-2"]
texts = ["Tyrion ate a bone.", "Daniel ate a sandwich.", "Tyrion climbed a tree.", "Tyrion is sleeping."]

In [ ]:
#no_test
#expected output: ['1-2']

entity = "Daniel"
entity_folio = "1"

genEntryIDs(folio_ids, entry_ids, texts, entity, entity_folio)

['1-2']

In [ ]:
#no_test
#expected output: ['1-1']

entity = "Tyrion"
entity_folio = "1"

genEntryIDs(folio_ids, entry_ids, texts, entity, entity_folio)

['1-1']

In [ ]:
#no_test
#expected output: ['1818-1','1818-2']

entity = "Tyrion"
entity_folio = "1818"

genEntryIDs(folio_ids, entry_ids, texts, entity, entity_folio)

['1818-1', '1818-2']

In [ ]:
#no_test
#expected output: []
#again, this (entities not appearing on the folio in question) shouldn't happen, but if it does we don't want to build a row
#in the final dataframe

entity = "Michael Jordan"
entity_folio = "1818"

genEntryIDs(folio_ids, entry_ids, texts, entity, entity_folio)

[]

## Exploring joining the data

The cells below use the functions defined above to combine the two input dataframes and generate spans. For the time being, they use the dummy data at the top of this document.  Any work using these functions requires the format of the dummy data.

In [ ]:
#no_test
#parsing dummy data (back) into lists
#start here with real data

vol_ids, vol_titls, fol_ids, entry_nos, entry_texts = parseXML(xml_df)
ent_types, ent_names, ent_fols = parseEntities(ent_df)

In [ ]:
#no_test
#building entry ID lists for entities

ent_entries = [[]] * len(ent_names)
index = 0

for entity in ent_names:    
    for folio in ent_fols[index]:        
        ent_entries[index] = ent_entries[index] + (genEntryIDs(fol_ids, entry_nos, entry_texts, entity, folio))        
    index += 1

In [ ]:
#no_test
#creating merged lists

out_volume_ids = []
out_volume_titles = []
out_folio_ids = []
out_entry_numbers = []
out_entry_texts = []
out_entity_names = []
out_span_starts = []
out_span_ends = []
out_labels = []

index = 0 #this is clumsy and could probably be implemented better with numpy, 
#but it's to protect against multiple instances of the same entity name

for entity in ent_names:
    for entry in ent_entries[index]:        
        folio_id = entry[:entry.find('-')]        
        volume_id = vol_ids[fol_ids.index(folio_id)]
        volume_title = vol_titls[fol_ids.index(folio_id)]        
        entry_text = entry_texts[entry_nos.index(entry)]        
        label = ent_types[ent_names.index(entity)]
        
        spans = generateSpans(entry_text, entity)        
        for span in spans:            
            out_volume_ids.append(volume_id)
            out_volume_titles.append(volume_title)
            out_folio_ids.append(folio_id)
            out_entry_numbers.append(entry)
            out_entry_texts.append(entry_text)
            out_entity_names.append(entity)
            out_span_starts.append(span[0])
            out_span_ends.append(span[1])
            out_labels.append(label)
        
    index += 1
    
collated_dict = {"vol_id": out_volume_ids, "vol_titl": out_volume_titles, "fol_id": out_folio_ids, "entry_no": out_entry_numbers, "text": out_entry_texts, "entity": out_entity_names, "start": out_span_starts, "end": out_span_ends, "label": out_labels}

In [ ]:
#no_test

collated_df = pd.DataFrame(collated_dict)
collated_df

,vol_id,vol_titl,fol_id,entry_no,text,entity,start,end,label
0,1,First book,102,102-1,Tyrion ate some food and Tyrion drank some water.,Tyrion,0,6,Person
1,1,First book,102,102-1,Tyrion ate some food and Tyrion drank some water.,Tyrion,25,31,Person
2,1,First book,102,102-2,Tyrion went to sleep while Daniel kept working.,Tyrion,0,6,Person
3,3,Third book,104,104-1,One day the rabbit would be eaten by Tyrion.,Tyrion,37,43,Person
4,1,First book,101,101-1,Daniel wrote some code.,Daniel,0,6,Person
5,1,First book,102,102-2,Tyrion went to sleep while Daniel kept working.,Daniel,27,33,Person
6,2,Second book,103,103-1,The rabbit ran away.,rabbit,4,10,Person
7,3,Third book,104,104-1,One day the rabbit would be eaten by Tyrion.,rabbit,12,18,Person


## Final code to generate joined dataframes

This function combines the previous four cells so that dataframe conversion can be performed in other notebooks.  `collate_frames` is the top-level function that should be called to join the dataframes.

In [ ]:
# export

def collate_frames(xml_df, ent_df):
    '''
    Function collate_frames: combines XML dataframe and entity dataframe to generate final dataframe with spans
        Inputs: xml_df: pandas DataFrame with columns "vol_id", 'vol_titl', 'fol_id', 'entry_no', and 'text'
                ent_df: pandas DataFrame with columns "Entity Type", "Name", and "Source Associator"    
        Outputs: a joined, collated dataframe of xml_df and ent_df
    '''
    #parsing input frames into lists
    vol_ids, vol_titls, fol_ids, entry_nos, entry_texts = parseXML(xml_df)
    ent_types, ent_names, ent_fols = parseEntities(ent_df)
    
    #building entry ID lists for entities
    ent_entries = [[]] * len(ent_names)
    index = 0

    for entity in ent_names:    
        for folio in ent_fols[index]:        
            ent_entries[index] = ent_entries[index] + (genEntryIDs(fol_ids, entry_nos, entry_texts, entity, folio))        
        index += 1   
        
    #creating collated lists
    out_volume_ids = []
    out_volume_titles = []
    out_folio_ids = []
    out_entry_numbers = []
    out_entry_texts = []
    out_entity_names = []
    out_span_starts = []
    out_span_ends = []
    out_labels = []

    index = 0 #this is clumsy and could probably be implemented better with numpy, 
    #but it's to protect against multiple instances of the same entity name

    for entity in ent_names:
        for entry in ent_entries[index]:            
            folio_id = entry[:entry.find('-')]        
            volume_id = vol_ids[fol_ids.index(folio_id)]
            volume_title = vol_titls[fol_ids.index(folio_id)]        
            entry_text = entry_texts[entry_nos.index(entry)]        
            label = ent_types[ent_names.index(entity)]

            spans = generateSpans(entry_text, entity)            
            for span in spans:            
                out_volume_ids.append(volume_id)
                out_volume_titles.append(volume_title)
                out_folio_ids.append(folio_id)
                out_entry_numbers.append(entry)
                out_entry_texts.append(entry_text)
                out_entity_names.append(entity)
                out_span_starts.append(span[0])
                out_span_ends.append(span[1])
                out_labels.append(label)

        index += 1
        
    #The loop below addresses a rare corner case that could result in tokens being mapped to multiple entities, which spaCy
    #does not allow. This occurs when one entity string in a given entry is a substring of another that appears in the same
    #entry.
    
    spanned_entries = [] #list of unique entries with entity spans
    spanned_tokens_per_entry = [] #list of indices that appear in spans for each entry
    i = 0
    while i < len(out_entry_numbers):
        if out_entry_numbers[i] in spanned_entries:            
            for k in range(out_span_starts[i], out_span_ends[i]):
                if k in spanned_tokens_per_entry[spanned_entries.index(out_entry_numbers[i])]:                    
                    del out_volume_ids[i]
                    del out_volume_titles[i]
                    del out_folio_ids[i]
                    del out_entry_numbers[i]
                    del out_entry_texts[i]
                    del out_entity_names[i]
                    del out_span_starts[i]
                    del out_span_ends[i]
                    del out_labels[i]
                    i -= 1
                    break
                else:
                    spanned_tokens_per_entry[spanned_entries.index(out_entry_numbers[i])].append(k)            
        else:
            spanned_entries.append(out_entry_numbers[i])
            temp = []
            for j in range(out_span_starts[i], out_span_ends[i]):
                temp.append(j)
            spanned_tokens_per_entry.append(temp)
        i += 1    
    
    collated_dict = {"vol_id": out_volume_ids, "vol_titl": out_volume_titles, "fol_id": out_folio_ids, "entry_no": out_entry_numbers, "text": out_entry_texts, "entity": out_entity_names, "start": out_span_starts, "end": out_span_ends, "label": out_labels}
    
    collated_df = pd.DataFrame(collated_dict)
    return collated_df


In [ ]:
# export

def manual_collate(xml_df, ent_csv):    
    #parsing input frames into lists
    vol_ids, vol_titls, fol_ids, entry_nos, entry_texts = parseXML(xml_df)
    ent_types, ent_names, ent_fols = parseManualEntities(ent_csv)
    
    #building entry ID lists for entities
    ent_entries = [[]] * len(ent_names)
    index = 0

    for entity in ent_names:    
        for folio in ent_fols[index]:        
            ent_entries[index] = ent_entries[index] + (genEntryIDs(fol_ids, entry_nos, entry_texts, entity, folio))        
        index += 1   
        
    #creating collated lists
    out_volume_ids = []
    out_volume_titles = []
    out_folio_ids = []
    out_entry_numbers = []
    out_entry_texts = []
    out_entity_names = []
    out_span_starts = []
    out_span_ends = []
    out_labels = []

    index = 0 #this is clumsy and could probably be implemented better with numpy, 
    #but it's to protect against multiple instances of the same entity name

    for entity in ent_names:
        for entry in ent_entries[index]:            
            folio_id = entry[:entry.find('-')]        
            volume_id = vol_ids[fol_ids.index(folio_id)]
            volume_title = vol_titls[fol_ids.index(folio_id)]        
            entry_text = entry_texts[entry_nos.index(entry)]        
            label = ent_types[ent_names.index(entity)]

            spans = generateSpans(entry_text, entity)            
            for span in spans:            
                out_volume_ids.append(volume_id)
                out_volume_titles.append(volume_title)
                out_folio_ids.append(folio_id)
                out_entry_numbers.append(entry)
                out_entry_texts.append(entry_text)
                out_entity_names.append(entity)
                out_span_starts.append(span[0])
                out_span_ends.append(span[1])
                out_labels.append(label)

        index += 1
        
    #The loop below addresses a rare corner case that could result in tokens being mapped to multiple entities, which spaCy
    #does not allow. This occurs when one entity string in a given entry is a substring of another that appears in the same
    #entry.
    
    spanned_entries = [] #list of unique entries with entity spans
    spanned_entities_per_entry = [] #list of unique entity name strings in each entry
    
    for entry_number in out_entry_numbers:
        if not (entry_number in spanned_entries):
            spanned_entries.append(entry_number)
            spanned_entities_per_entry.append([])            
    
    for i in range(len(out_entity_names)):
        entry_pos = spanned_entries.index(out_entry_numbers[i])
        if not (out_entity_names[i] in spanned_entities_per_entry[entry_pos]):
            spanned_entities_per_entry[entry_pos].append(out_entity_names[i])            
    
    output_pos = 0
    while output_pos < len(out_entity_names):
        entity_entry = spanned_entries.index(out_entry_numbers[output_pos])
        for spanned_entity in spanned_entities_per_entry[entity_entry]:
            if (out_entity_names[output_pos] in spanned_entity) and (out_entity_names[output_pos] != spanned_entity):
                del out_volume_ids[output_pos]
                del out_volume_titles[output_pos]
                del out_folio_ids[output_pos]
                del out_entry_numbers[output_pos]
                del out_entry_texts[output_pos]
                del out_entity_names[output_pos]
                del out_span_starts[output_pos]
                del out_span_ends[output_pos]
                del out_labels[output_pos]
                output_pos -= 1
                break
        output_pos += 1
    
    collated_dict = {"vol_id": out_volume_ids, "vol_titl": out_volume_titles, "fol_id": out_folio_ids, "entry_no": out_entry_numbers, "text": out_entry_texts, "entity": out_entity_names, "start": out_span_starts, "end": out_span_ends, "label": out_labels}
    
    collated_df = pd.DataFrame(collated_dict)
    return collated_df

In [ ]:
gh = [1,2,3,4,4,5]

for element in gh:
    print(gh.index(element))

0
1
2
3
3
5


In [ ]:
#no_test

collate_frames(xml_df, ent_df)

,vol_id,vol_titl,fol_id,entry_no,text,entity,start,end,label
0,1,First book,102,102-1,Tyrion ate some food and Tyrion drank some water.,Tyrion,0,6,Person
1,1,First book,102,102-1,Tyrion ate some food and Tyrion drank some water.,Tyrion,25,31,Person
2,1,First book,102,102-2,Tyrion went to sleep while Daniel kept working.,Tyrion,0,6,Person
3,3,Third book,104,104-1,One day the rabbit would be eaten by Tyrion.,Tyrion,37,43,Person
4,1,First book,101,101-1,Daniel wrote some code.,Daniel,0,6,Person
5,1,First book,102,102-2,Tyrion went to sleep while Daniel kept working.,Daniel,27,33,Person
6,2,Second book,103,103-1,The rabbit ran away.,rabbit,4,10,Person
7,3,Third book,104,104-1,One day the rabbit would be eaten by Tyrion.,rabbit,12,18,Person


## Generating Spacy input format from dataframes

Spacy requires the data to be a list of tuples.  Each tuple should contain the text as a string, and a dictionary.  The dictionary should have the word 'entities' as a key, and the values should be a list of tuples which contain information about the entities.  This information will be of the form: `(span_start, span_end, Entity_type)`.  The `genSpacyInput` function performs this functionality using the dataframe as input.

In [ ]:
# export

def genSpaCyInput(df):
    '''
    Function genSpaCyInput:function takes a dataframe with columns/contents as above and transforms it into a list of tuples, each consisting of raw text 
    paired with a dictionary of entities that appear in that text, for input into spaCy. 
        Inputs: df: dataframe with columns 'text', 'label', 'start', 'end'
        Outputs: data in SpaCy format
    '''
    text = df["text"].tolist()
    label = df["label"].tolist()
    start = df["start"].tolist()
    end = df["end"].tolist()    
    
    #build list of unique entries and list of empty annotation dictionaries for each
    u_txt = []
    annot_ls = []
    
    for tx in text:
        if not (tx in u_txt):
            u_txt.append(tx)
            annot_ls.append({"entities":[]})
            
    #populate annotation dictionaries
    for i in range(len(label)):
        pos = u_txt.index(text[i])        
        annot_ls[pos]["entities"].append((int(start[i]), int(end[i]), label[i]))       
        
    #build list of tuples
    tuples = []    
    for i in range(len(u_txt)):
        tuples.append((u_txt[i], annot_ls[i]))       
    
    return tuples

In [ ]:
#no_test

genSpaCyInput(collated_df)

[('Tyrion ate some food and Tyrion drank some water.',
  {'entities': [(0, 6, 'Person'), (25, 31, 'Person')]}),
 ('Tyrion went to sleep while Daniel kept working.',
  {'entities': [(0, 6, 'Person'), (27, 33, 'Person')]}),
 ('One day the rabbit would be eaten by Tyrion.',
  {'entities': [(37, 43, 'Person'), (12, 18, 'Person')]}),
 ('Daniel wrote some code.', {'entities': [(0, 6, 'Person')]}),
 ('The rabbit ran away.', {'entities': [(4, 10, 'Person')]})]

## Hyper-efficient df to Spacy format

It's pretty unassuming, but this is actually the function that just takes the two frames and returns spaCy input...

In [ ]:
#export

def frames_to_spacy(xml_df, ent_df):
    df = collate_frames(xml_df, ent_df)
    return genSpaCyInput(df)

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 31-collate-xml-entities-spans.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted data-preprocessing.ipynb.
Converted markup-to-spacy.ipynb.
Converted unstructured-to-markup.ipynb.
